# Práctica 2: Aprendizaje y selección de modelos de clasificación*

### Minería de Datos: Curso académico 2020-2021

### Alumnos:

* José Luis Bernáldez Morales
* Guillermo López Bermejo

Esta práctica es una continuación de la realizada sobre los conjuntos de datos *pima_diabetes* y *wisconsin_breast_cancer*. Ahora, nos centraremos en los distintos modelos que podemos utilizar, la mayoría ensembles, y en la configuración de sus hiperparámetros con el objetivo de encontrar la configuración óptima para la resolución de nuestro problema.

In [ ]:
# Comencemos con los imports
from sklearn.base import clone
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer
import pandas as pd



import miner_a_de_datos_aprendizaje_modelos_utilidad as utils

Y ahora definiremos la semilla para poder reproducir los resultados.

In [ ]:
random_state = 27912

## 2. Modelos de clasificación supervisada

En primer lugar vamos a definir nuestras dos bases de datos:

### Breast Cancer Wisconsin

Si recordamos de la práctica anterior, este dataset se compone de 569 muestras y se utiliza para el problema de clasificación de si un tumor es benigno o maligno. Cada una de las instancias se compone de la variable clase, la id, una variable Unname: 32 que tendremos que eliminar y 30 variables predictoras, divididas en valores medios, el error y el peor valor. Estas variables predictoras son:

* Radius: media de las distancias del centro al perímetro
* Texture: desviación estandar de valores en escala de grises
* Perimeter
* Area
* Smoothness: variación local en la longitud de los radio
* Compactness: perímetro^2 / area - 1
* Concavity: severidad de las porciones cóncavas del contorno
* Concave points
* Symmetry
* Fractal dimension: "coastline approximation" - 1

### Pima Indians Diabetes

Se trata de una base de datos creada por el *Instituto Nacional de Enfermedades Digestivas y de Riñón* en la que se intenta predecir si un paciente tiene diabetes o no basándose en los siguientes parámetros:

* Pregnancies: número de embarazos
* Glucose: Concentración de glucosa en plasma a 2 horas en una prueba de tolerancia a la glucosa oral
* BloodPressure: presión sanguínea medida en mm Hg
* SkinThickness: espesor del pliegue de la piel del tríceps
* Insulin: insulina en sangre en mU/mL
* BMI: índice de masa corporal medido en peso en kg/(altura en m)^2
* DiabetesPedigreeFunction: índice que repredenta una síntesis de la historia de diabetes mellitus en familiares y la relación genética de esos familiares con el sujeto
* Age: edad
* Outcome: variable clase: 0 si no tiene diabetes y 1 si la tiene

## 1. Carga de datos

Ahora, vamos a proceder a cargar la base de datos, así como a separar nuestro dataset en un conjunto de entrenamiento y test, mediante un holdout, tal y como hicimos en la práctica anterior.

In [ ]:
filepath = "../input/breast-cancer-wisconsin-data/data.csv"

index = "id"
target = "diagnosis"   # Nuestra variable objetivo

data = utils.load_data(filepath, index, target)
data.sample(5, random_state = random_state)

Podemos ver que hemos cargado los datos correctamente. De nuevo, el fichero csv tiene una coma al final de la fila donde se declaran las columnas, generando una variable extra sin nombre, que es Unnamed: 32, la cual debe ser eliminada porque no nos aporta información. Además, como vamos a utilizar otras métricas a parte del accuracy, vamos a cambiar la variable clase, que pasará a ser 0 y 1 para que dichas métricas funcionen correctamente.

In [ ]:
data = data.drop('Unnamed: 32', axis=1)
data['diagnosis']=data['diagnosis'].map({'M':1,'B':0})

In [ ]:
filepath2 = "../input/pima-indians-diabetes-database/diabetes.csv"

target2 = "Outcome"

data2 = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
data2[target2] = data2[target2].astype("category")
data2.sample(5, random_state = random_state)

Ahora dividimos el dataset:

In [ ]:
(X, y) = utils.divide_dataset(data, target)
X.sample(5, random_state=random_state)

In [ ]:
(X2, y2) = utils.divide_dataset(data2, target2)
X2.sample(5, random_state=random_state)

Por último, aplicaremos un holdout estratificado para asegurarnos de mantener la distribución de la variable clase en las particiones del conjunto de datos.

In [ ]:
stratify = y
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=stratify,
                                                      train_size=train_size,
                                                      random_state=random_state)
X_train.sample(5, random_state=random_state)

In [ ]:
stratify = y2
train_size = 0.7

(X2_train, X2_test, y2_train, y2_test) = train_test_split(X2, y2,
                                                      stratify=stratify,
                                                      train_size=train_size,
                                                      random_state=random_state)
X2_train.sample(5, random_state=random_state)

## 2. Modelos de clasificación supervisada

En este apartado nos centraremos en explicar cuáles serán los modelos de clasificación que vamos a usar para esta práctica. Entre ellos se encuentran los árboles de decisión que tantas veces hemos visto, el algoritmo de los vecinos más cercanos y diferentes ensembles. Estos ensembles son combinaciones de clasificadores, es decir, son modelos aprenden un conjunto de clasificadores a partir de los datos y clasifican en función de la importancia que tiene cada uno de los clasificadores o en base a la agregación de las predicciones individuales de cada uno.

### 2.1. Vecinos más cercanos

El algoritmo de los vecinos más cercanos se encuentra dentro del paradigma perezoso de aprendizaje. Esto significa que, en vez de contruir un modelo, la base de datos es el propio modelo o conjunto de entrenamiento. Por lo tanto, se trabaja cuando llega un nuevo caso a clasificar y se buscan las instancias más parecidas para clasificar como tal.

El clasificador se implementa en la clase `neighbors.KNeighborsClassifier`. Los hiperparámetros principales son:

* `n_neighbors`: Se trata del número de vecinos más cercanos (por defecto a 5). Este hiperparámetro es de vital importancia, pues un número muy bajo de vecinos provocará un sobreajuste, además de que el algoritmo se vuelve muy sensible al ruido.Por otro lado, cuanto mayor sea en número de vecinos a tener en cuenta, más se asemejará el comportamiento del algoritmo a un `zero_r`

* `weights`: Generalmente, el peso de los vecinos es siempre el mismo, de manera que todos tienen la misma importancia. No obstante, podemos pesar los registros con la inversa de la distancia para minimizar el error a la hora de clasificar.

Vamos a inicializar el clasificador con un número moderado de vecinos.

In [ ]:
n_neighbors = 10
k_neighbors_model = KNeighborsClassifier(n_neighbors)

### 2.2. Árboles de decisión

Un árbol de decisión representa la función de hipótesis mediante grafo dirigido tal que:
* Cada nodo representa una varaible de entrada
* Cada rama pende de un nodo representa uno de los posibles valores que puede tomar la variable correspondiente.
* Las hojas corresponden con valores de la variable de clase.
* Un ejemplo se clasifica recorriendo el árbol desde la raíz y eligiendo en cada momento la rama que satisface la condición para el valor del atributo correspondiente. La clase elegida sería la asignada a la hoja a la que se llega.

Un árbol de decisión también puede verse como un sistema de reglas. Se encuentran implementados en la clase `tree.DecisionTreeClassifier`. Los hiperparámetros principales son:

* `criterion`: Criterio utilizado para seleccionar la calidad de una partición.
* `max_depth`: Profundidad máxima a la que dejamos que el árbol se ramifique.
* `random_state`: Semilla para controlar la reproducibilidad de los experimentos (por defecto, `random_state=None`).
* `ccp_alpha`: Parámetro de complejidad usado para el algoritmo de post-poda *Minimal Cost-Complexity Pruning* (por defecto, `ccp_alpha=0.0`).

Los árboles con hiperparámetros por defecto suelen estar sobreajustados, puesto que no tiene profundidad máxima y permitimos que una hoja tenga únicamente un ejemplo. De aquí la importancia de configurar estos hiperparámetros.

Inicialicemos un árbol de decisión por defecto (totalmente sobreajustados):

In [ ]:
decision_tree_model = DecisionTreeClassifier(random_state=random_state)

### 3.3. Adaptative Boosting (AdaBoost)

AdaBoost será el primer ensemble con el que trabajaremos, el cual es una versión del algoritmo de boosting. En este tipo de algoritmos se asignan pesos a cada ejemplo del conjunto de entrenamiento, y se aprende de manera secuencial (0 paralelismo) un conjunto de T clasificadores (por ejemplo, árboles de profundidad limitada). Para el aprendiazje de cada modelo, se unas los pesos asociados a las instancias, y después de que se aprenda el modelo, se incrementan los pesos de aquellos ejemplos del conjunto de entrenamiento clasificados de manera incorrecta. Luego, el siguiente modelo se aprende con los pesos actualizados.

El modelo final agrega los modelos y combina los votos de cada clasificador de manera individual, donde el peso de cada clasificador se define en función de su precisión. Nótese que podría darse el caso de que los modelos sobreajusten a datos mal clasificados, pero ésto solo se da si nos quedamos cortos de iteraciones o si el error aumenta demasiado como para poder corregirlo.

Esta técnica utiliza clasificadores conocidos como *weak learners*, los cuales tienen poco poder de generalización y parámetros simples. Sin embargo, a lo largo de las iteraciones, el algoritmo optimiza estos parámetros para convertir a los clasificadores en *strong learners*.

El algoritmo se implementa en la clase `ensemble.AdaBoostClassifier`. Los hiperparámetros principales son:

* `base_estimator`: Estimador base que usa el ensemble (por defecto, `base_estimator=None`). Si no se especifica, se utiliza un árbol de decisión de profundidad uno.
* `n_estimators`: Número de estimadores a aprender (por defecto, `n_estimators=50`).
* `learning_rate`: Parámetro de regularización para controlar la contribución de cada estimador (por defecto, `learning_rate=1.0`).
* `random_state`: Semilla para controlar la reproducibilidad de los experimentos (por defecto, `random_state=None`).

Vamos a configurar un modelo *AdaBoost* sencillo que utilice los hiperparámetros por defecto:

In [ ]:
adaboost_model = AdaBoostClassifier(random_state=random_state)

### 3.4. Bootstrap Aggregating (Bagging)

En el algoritmos de Bootstrap Aggregating, dado un conjunto *D* de n ejemplos, en cada iteración *i* de obtiene un conjunto *Di*, también de tamaño n, mediante muestreo aleatorio con reemplazo de *D*. De esta manera tenemos tantos conjuntos de entrenamiento como queramos, de los cuales aprenderemos un modelo de clasificación a partir de cada uno de ellos (podemos aplicar paralelismo).

Cada uno de estos clasificadores devolverá una predicción, y mediante voto por mayoría, se realizará la clasificación. Si nuestro problema es de regresión o predicción numérica, la clasificación se realizará por la media de los valores devueltos por cada modelo.

Esta estrategia aprovecha los diferentes conjuntos de datos para generar modelos diversos, reduciendo así el error obtenido mediante varianza. Para conseguir esto, los clasificadores deben estar lo menos correlacionados entre sí que se pueda, y de ahí el uso de muestreo, ya que la aleatorización disminuirá la correlación entre los clasificadores (teóricamente, puesto que será muy dificil que no exista ninguna correlación entre los clasificadores).

Esta técnica utiliza de base clasificadores tipo *strong learners* puesto que poseen mucha varianza, y el objetivo es reducirla a medida que se agregan las predicciones de cada uno de los clasificadores.

El algoritmo se implementa en la clase `ensemble.BaggingClassifier`. Los hiperparámetros principales son:

* `base_estimator`: Estimador base que usa el ensemble (por defecto, `base_estimator=None`). Si no se especifica, se utiliza un árbol de decisión sin podar.
* `n_estimators`: Número de estimadores a aprender (por defecto, `n_estimators=10`).
* `max_samples`: Número o proporción de muestras a utilizar en el muestreo (por defecto, `max_samples=1.0`).
* `max_features`: Número o proporción de características a utilizar en el muestreo (por defecto, `max_features=1.0`).
* `bootstrap`: Cuándo el muestreo de instancias es con reemplazo (por defecto, `bootstrap=True`).
* `bootstrap_features`: Cuándo el muestreo de características es con reemplazo (por defecto, `bootstrap_features=False`).
* `random_state`: Semilla para controlar la reproducibilidad de los experimentos (por defecto, `random_state=None`).

Vamos a configurar un ensemble tipo *Bagging* básico (hiperparámetros por defecto):

In [ ]:
bagging_model = BaggingClassifier(random_state=random_state)

### 3.5. Random Forests

Random Forests es un ensemble formado por la combinación de modelos con forma de árbol. Cada uno de estos árboles se construye de manera independiente y **semi-aleatoria**, componente que reduce la correlación entre los diferentes modelos. Además, los árboles no se suelen podar aunque a veces se limita su tamaño.

La aleatorización en la construcción de los árboles se consigue seleccionando aleatoriamente m de los p atributos (m << p), evaluarlos y escoger el mejor para formar un nodo. Esta evaluación consiste en calcular su ganancia de información. Los árboles suelen ser más grandes que los que construiría C4.5, pero ahorra bastante el tiempo de proceso en la construcción de cada árbol, ya que sólamente se evalúa un subconjunto de atributos en cada nodo interno.

El ensemble *Random Forests* se implementa en la clase `ensemble.RandomForestClassifier` y recopilando los hiperparámetros más importantes tenemos:

* `n_estimators`: Número de estimadores a aprender (por defecto, `n_estimators=100`).
* `criterion`: Criterio utilizado para medir la calidad de una partición (por defecto, `criterion="gini"`).
* `max_features`: Número de características a considerar en cada nodo del árbol (por defecto, `max_features="auto"`).
* `random_state`: Semilla para controlar la reproducibilidad de los experimentos (por defecto, `random_state=None`).

Este ensemble es bastante interesante, puesto que realmente solo tiene 2 parámetros interesantes, son el número de estimadores y el número de variables que van a considerar en cada nodo los árboles, puesto que los otros dos no son tan relevantes para el ensemble. Es decir, es un ensemble que no requiere de un preprocesamiento muy complejo, además de que es bastante preciso, no produce sobreajuste cuando se utiliza un número alto de árboles, el número de características a considerar no es necesario ajustarlo con mucha precisión y son muy eficientes en tareas de predicción (gracias al paralelismo).  

Vamos a configurar un estimador tipo *Random Forests* muy básico usando los hiperparámetros por defecto:

In [ ]:
random_forest_model = RandomForestClassifier(random_state=random_state)

### 3.6. Gradient Tree Bosting (Gradient Boosting)

Gradient Boosting es una versión del algoritmo Boosting que hemos visto anteriormente. Esto quiere decir que es un algoritmo iterativo (no podemos aplicar paralelismo), en el cual se utilizan los residuos (gradiente negativo) de todas las instancias para mejorar la aproximación. En cada iteración, se usan todos los modelos aprendidos previamente, y todos los modelos tienen la misma importancia, tanto en aprendizaje como en inferencia.

El caso de Gradient Tree Boosting es una generalización de Boosting con la capacidad de optimizar cualquier función de pérdida, y permite resolver otro tipo de problemas como los de regresión.

Este algoritmo está implementado en la clase `ensemble.GradientBoostingClassifier` y los hiperparámetros más importantes son:

* `loss`: Función de pérdida a optimizar (por defecto, `loss="deviance"`).
* `learning_rate`: Parámetro de regularización para controlar la contribución de cada estimador (por defecto, `learning_rate=0.1`).
* `n_estimators`: Número de estimadores a aprender (por defecto, `n_estimators=100`).
* `criterion`: Criterio utilizado para medir la calidad de una partición (por defecto, `criterion="friedman_mse"`).
* `max_depth`: Altura máxima de los árboles de decisión (por defecto, `max_depth=3`).
* `ccp_alpha`: Parámetro de complejidad usado para el algoritmo de post-poda *Minimal Cost-Complexity Pruning* (por defecto, `ccp_alpha=0.0`).
* `random_state`: Semilla para controlar la reproducibilidad de los experimentos (por defecto, `random_state=None`).

Vamos a configurar este estimador usando los hiperparámetros por defecto:

In [ ]:
gradient_boosting_model = GradientBoostingClassifier(random_state=random_state)

### 3.7. Histogram-Based Gradient Boosting (Histogram Gradient Boosting)

El algoritmo *Histogram-Based Gradient Boosting (Histogram Gradient Boosting)* optimiza el algoritmo de Gradient Boosting, siendo éste una versión mucho más rápida. Para conseguir dicha mejora, el algoritmo discretiza el conjunto de datos de entrada para reducir el número de puntos de corte a considerar en las construcción de los árboles de decisión.

El algoritmo *Histogram Gradient Boosting* se implementa en la clase `ensemble.HistGradientBoostingClassifier` y cuyos hiperparámetros más básicos son:

* `loss`: Función de pérdida a optimizar (por defecto, `loss="auto"`).
* `learning_rate`: Parámetro de regularización para controlar la contribución de cada estimador (por defecto, `learning_rate=0.1`).
* `max_iter`: Número máximo de iteraciones del algoritmo (por defecto, `max_iter=100`).
* `max_leaf_nodes`: Número máximo de nodos hoja (por defecto, `max_leaf_nodes=31`).
* `max_depth`: Altura máxima de los árboles de decisión (por defecto, `max_depth=None`).
* `random_state`: Semilla para controlar la reproducibilidad de los experimentos (por defecto, `random_state=None`).

Vamos a usar los hiperparámetros por defecto para configurar este estimator:

---

**Nota**: Este estimador es todavía experimental y se tiene que activar explícitamente con `experimental.enable_hist_gradient_boosting`.

---

In [ ]:
hist_gradient_boosting_model = HistGradientBoostingClassifier(random_state=random_state)

Finalmente, para que sea más sencillo y por evitar cualquier tipo de data leak, vamos a renombrar las variables de los conjuntos de entrenamiento.

In [ ]:
X = X_train
y = y_train

X2 = X2_train
y2 = y2_train

## 3. Evaluación de modelos

Ahora ya conocemos mejor cuáles son los modelos que tenemos que probar, por lo que nuestro objetivo será ver cuál podría ser el mejor clasificador para nuestro problema. Para ello, utilizaremos una validación cruzada.

La validación cruzada es una técnica muy utilizada con la que nos garantizamos que los resultados de las pruebas que vamos a realizar no están sesgados. Esta técnica consiste en separar el conjunto de datos en *k* particiones, de las cuales elegiremos una de ellas como conjunto de test y realizaremos un aprendizaje con el resto de particiones. Este proceso de repite *k* veces, de manera que los conjuntos de entrenamiento y test son siempre distintos.

Cuando hemos terminado el proceso, el resultado es la media de las métricas obtenidas para cada partición. La forma de implementarlo será la siguiente:

In [ ]:
n_splits = 10
n_repeats = 5

cv = RepeatedStratifiedKFold(n_splits=n_splits,
                             n_repeats=n_repeats,
                             random_state=random_state)

Puesto que en la práctica anterior nos dedicamos a realizar un preprocesamiento de los datos, vamos a aprovechar dicho trabajo para que nuestros modelos funcionen aún mejor. Para esto, volveremos a realizar los cambios necesarios al conjunto de datos de entrenamiento (eliminación de algunas de las variables) y los vamos a aplicar a nuestro conjunto de datos mediante un pipeline.

### Breast Cancer Wisconsin

In [ ]:
# Estas son todas las variables a eliminar
variables_mean_drop = ['perimeter_mean','area_mean','concave points_mean']
variables_se_drop = ['perimeter_se','area_se','concave points_se']
variables_worse_drop = ['perimeter_worst','area_worst','concave points_worst']

# En la práctica anterior vimos que la discretización era incorrecta, por lo que hemos decidido no incluirla en el pipeline.

reduccion_datos = variables_mean_drop + variables_se_drop + variables_worse_drop
elim_var = make_column_transformer(("drop", reduccion_datos), remainder="passthrough")

### Pima Indians Diabetes

In [ ]:
impute = make_pipeline(SimpleImputer(strategy="median", missing_values=0))

columns = 'Glucose|BloodPressure|SkinThickness|BMI|DiabetesPedigreeFunction'
non = 'Pregnancies'

imputer = make_column_transformer(
    (impute, make_column_selector(pattern=columns)),
    ('passthrough', make_column_selector(pattern=non))
)

Ahora, solo nos queda ir probando cada clasificador para ver como se comportan a la hora de resolver nuestro problema. Como ya sabemos, estamos ante un problema de clasificación relacionado con la salud donde lo que queremos es identificar el mayor número de casos malignos, puesto que pasarlos por alto supondría un riego para la vida de otras personas. Es por esto que no solo vamos a fijarnos en la tasa de acierto de cada clasificador, sino que nos interesa conocer el recall de los diferentes algoritmos, puesto que es el parámetro que queremos maximizar.

Es por esto que vamos a definir ya las métricas que vamos a usar a lo largo de la libreta.

In [ ]:
scoring = ['accuracy', 'recall']
refit="recall"  # Para que el mejor clasificador se decida en función del recall

### Vecinos más cercanos

### Breast Cancer Wisconsin

In [ ]:
model_neighbors = k_neighbors_model
pipeline_neighbors = make_pipeline(elim_var, model_neighbors)
utils.evaluate_estimator(pipeline_neighbors, X, y, cv)

El algoritmo de los vecinos más cercanos funciona de manera bastante correcta, gracias al uso de un valor moderado en el número de vecinos que se tienen en cuenta para cada clasificación.

In [ ]:
utils.metrics(pipeline_neighbors,
               X_train, X_test,
               y_train, y_test)

No obstante, vemos que el recall es de 0.78, o lo que es lo mismo, solo tenemos un 78% de verdaderos positivos. Al tratarse de un problema de diagnóstico, lo que queremos es que haya el mínimo número de falsos negativos, por lo que este clasificador no es del todo bueno para esta tarea.

### Pima Indians Diabetes

In [ ]:
pipeline_neighbors2 = make_pipeline(imputer, model_neighbors)
utils.evaluate_estimator(pipeline_neighbors2, X2, y2, cv)

In [ ]:
utils.metrics(pipeline_neighbors2,
               X2_train, X2_test,
               y2_train, y2_test)

Podemos ver que en cuanto al accuracy, no obtenemos un valor demasiado prometedor, pero el recall obtenido sí que está muy por debajo de lo que cabría esperar para resolver un problema de clasificación como este.

### Árboles de decisión

### Breast Cancer Wisconsin

In [ ]:
model_trees = decision_tree_model
pipeline_trees = make_pipeline(elim_var, model_trees)
utils.evaluate_estimator(pipeline_trees, X, y, cv)

Tal y como se vio en la práctica anterior, los árboles son una buena herramienta para la clasificación. Podemos observar por el valor del accuracy en el conjunto de entrenamiento que los árboles se han sobreajustado al máximo, lo cual provoca una bajada en dicho accuracy cuando nos enfrentamos al conjunto de test.

In [ ]:
utils.metrics(pipeline_trees,
               X_train, X_test,
               y_train, y_test)

En cuanto al recall, vemos que es bastante mejor que en el caso de los vecinos más cercanos, reduciendo así el número de falsos negativos.

### Pima Indians Diabetes

In [ ]:
pipeline_trees2 = make_pipeline(imputer, model_trees)
utils.evaluate_estimator(pipeline_trees2, X2, y2, cv)

In [ ]:
utils.metrics(pipeline_trees2,
               X2_train, X2_test,
               y2_train, y2_test)

Podemos ver que los árboles de decisión parecen comportarse peor incluso si lo medimos por el accuracy obtenido. Aún así, el recall ha mejorado un poco, aunque sigue siendo bastante bajo.

### Adaptative Boosting (AdaBoost)

### Breast Cancer Wisconsin

In [ ]:
model_adaboost = adaboost_model
pipeline_adaboost = make_pipeline(elim_var, model_adaboost)
utils.evaluate_estimator(pipeline_adaboost, X, y, cv)

AdaBoost es el primer ensemble que probamos, y como era de esperar, su rendimiento es superior al de los anteriores clasificadores. Podemos ver una tasa de acierto bastante alta en el conjunto de test aun con un modelo que se ha sobreajustado completamente a los valores de entrenamiento.

In [ ]:
utils.metrics(pipeline_adaboost,
               X_train, X_test,
               y_train, y_test)

El recall obtenido de este clasificador es bastante bueno. Podemos ver que aún así hay ciertos casos que se nos pueden escapar, pero no parece un mal punto de partida para la resolución de este problema.

### Pima Indians Diabetes

In [ ]:
pipeline_adaboost2 = make_pipeline(imputer, model_adaboost)
utils.evaluate_estimator(pipeline_adaboost2, X2, y2, cv)

In [ ]:
utils.metrics(pipeline_adaboost2,
               X2_train, X2_test,
               y2_train, y2_test)

De momento, el resultado obtenido por el ensemble es el mejor hasta el momento. No solo ha aumentado el accuracy, sino que el recall, que es la métrica que estamos buscando maximizar, también ha aumentado.

### Bootstrap Aggregating (Bagging)

### Breast Cancer Wisconsin

In [ ]:
model_bagging = bagging_model
pipeline_bagging = make_pipeline(elim_var, model_bagging)
utils.evaluate_estimator(pipeline_bagging, X, y, cv)

En cuanto al Bootstrap Aggregating, hemos obtenido un rendimiento algo por debajo del ensemble anterior. No obstante, este clasificador sobreajusta menos y se ejecuta en un tiempo mucho menor, gracias al paralelismo.

In [ ]:
utils.metrics(pipeline_bagging,
               X_train, X_test,
               y_train, y_test)

Como habíamos previsto, los valores del recall para este clasificador también son peores, por lo que tendremos que ver si nos compensa ganar en tiempo de ejecución a costa de perder en recall.

### Pima Indians Diabetes

In [ ]:
pipeline_bagging2 = make_pipeline(imputer, model_bagging)
utils.evaluate_estimator(pipeline_bagging2, X2, y2, cv)

In [ ]:
utils.metrics(pipeline_bagging2,
               X2_train, X2_test,
               y2_train, y2_test)

Aquí podemos ver como el Bagging nos proporciona unos resultados muy similares a AdaBoost, pero con un tiempo de ejecución mucho mejor.

### Random Forests

### Wisconsin Breast Cancer

In [ ]:
model_rf = random_forest_model
pipeline_rf = make_pipeline(elim_var, model_rf)
utils.evaluate_estimator(pipeline_rf, X, y, cv)

Random Forest es el clasificador que mejor rendimiento nos ha dado por el momento. El tiempo de ejecución es algo mayor que el de otros ensembles, y los árboles de decisión que utiliza se sobreajustan por completo al conjunto de datos de entrenamiento, pero los resultados que produce son muy buenos.

In [ ]:
utils.metrics(pipeline_rf,
               X_train, X_test,
               y_train, y_test)

Efectivamente, el recall de este clasificador está muy próximo al 100%, por lo que nos interesa utilizarlo para nuestro problema.

### Pima Indians Diabetes

In [ ]:
pipeline_rf2 = make_pipeline(imputer, model_rf)
utils.evaluate_estimator(pipeline_rf2, X2, y2, cv)

In [ ]:
utils.metrics(pipeline_rf2,
               X2_train, X2_test,
               y2_train, y2_test)

Random Forests a priori parece de las mejores soluciones también, dándonos los valores más altos en ambas métricas.

### Gradient Tree Boosting (Gradient Boosting)

### Wisconsin Breast Cancer

In [ ]:
model_gradient = gradient_boosting_model
pipeline_gradient = make_pipeline(elim_var, model_gradient)
utils.evaluate_estimator(pipeline_gradient, X, y, cv)

Gradient Tree Boosting también clasifica de manera bastante buena a pesar de sobreajustar al máximo el conjunto de entrenamiento.

In [ ]:
utils.metrics(pipeline_gradient,
               X_train, X_test,
               y_train, y_test)

El recall obtenido es bueno, pero sigue sin ser superior al de Random Forests.

### Pima Indians Diabetes

In [ ]:
pipeline_gradient2 = make_pipeline(imputer, model_gradient)
utils.evaluate_estimator(pipeline_gradient2, X2, y2, cv)

In [ ]:
utils.metrics(pipeline_gradient2,
               X2_train, X2_test,
               y2_train, y2_test)

Gradient Tree Boosting ha mejorado algo el recall a costa de perder un poco de accuracy.

### Histogram-Based Gradient Boosting (Histogram Gradient Boosting)

### Wisconsin Breast Cancer

In [ ]:
model_h_gradient = hist_gradient_boosting_model
pipeline_h_gradient = make_pipeline(elim_var, model_h_gradient)
utils.evaluate_estimator(pipeline_h_gradient, X, y, cv)

Como sabemos, Histogram-Based Gradient Boosting es una mejora del ensemble anterior, tanto en tiempo de ejecución como en score, como podemos ver en la tabla.

In [ ]:
utils.metrics(pipeline_h_gradient,
               X_train, X_test,
               y_train, y_test)

El recall obtenido es igual al de Random Forests, por lo que también es un claro candidato a ser el mejor clasificador para el problema.

Una vez analizados todos los algoritmos, podemos ver que, a priori, el ensemble Random Forests puede ser nuestra mejor opción para el dataset de Wisconsin. No obstante, esta comparativa la hemos realizado con los modelos por defecto, por lo que los resultados no son concluyentes. Para ello, tendremos que analizar los algoritmos de nuevo en el siguiente apartado.

### Pima Indians Diabetes

In [ ]:
pipeline_h_gradient2 = make_pipeline(imputer, model_h_gradient)
utils.evaluate_estimator(pipeline_h_gradient2, X2, y2, cv)

In [ ]:
utils.metrics(pipeline_h_gradient2,
               X2_train, X2_test,
               y2_train, y2_test)

Podemos ver que, a pesar de ser más rapido que su versión sin histogramas, este Histogram Gradient Boosting funciona algo peor.

## 4. Selección de modelos

Ya hemos visto los diferentes clasificadores que tenemos, y podemos hacernos una idea de cuál será su rendimiento. No obstante, y como hemos comentado, no podemos analizar la eficiencia de los clasificadores para nuestro problema sin adecuar correctamente los parámetros.

En este apartado, utilizaremos un algoritmo de selección de modelos por fuerza bruta llamado Grid Search, el cual está implementado en `model_selection.GridSearchCV`. Este algoritmo recibe un grid de hiperparámetros que nosotros tendremos que seleccionar, y realiza una búsqueda exhaustiva evaluando mediante validación cruzada todas las posibles combinaciones entre los parámetros del clasificador para poder seleccionar los mejores valores para los hiperparámetros de los clasificadores.

Una vez obtengamos los estimadores con los mejores valores para sus hiperparámetros, los volveremos a evualuar, para ver si la estimación de los clasificadores que hemos realizado al principio cambia mucho o no.

### Vecinos más cercanos

Comenzando por el algoritmo de los vecinos más cercanos, los hiperparámetros que a priori parecen más importantes son el número de vecinos y la función de pesado.

* `weights`: utilizaremos tanto *uniform*, donde los puntos tienen el mismo peso, y *distance*, donde los puntos se pesan por la inversa de la distancia.
* `n_neighbors`: utilizaremos valores de entre 5 a 15, puesto que menos de 5 es demasiado poco, ya que el algoritmo sobreajustaría mucho y sería muy sensible al ruido, y más de 15 son demasiados vecinos y las fronteras podrían empezar a cruzarse y clasificaría mal.

### Breast Cancer Wisconsin

In [ ]:
estimator = pipeline_neighbors


weights = ["uniform", "distance"]
n_neighbors = [5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

k_neighbors_clf = utils.optimize_params(estimator,
                                       X, y, cv,scoring=scoring,refit=refit,
                                       kneighborsclassifier__weights=weights,
                                       kneighborsclassifier__n_neighbors=n_neighbors)

Podemos ver que el valor de 8 vecinos es un número razonable para no sobreajustar pero para clasificar de la mejor manera posible. Además, utilizar la inversa de la distancia para pesar los puntos es una buena idea puesto que penaliza mucho las distancias grandes, por lo que los vecinos estarían muy próximos entre si.

La mejor tasa de acierto obtenida es de 0.934 y el mejor recall es de 0.89.

### Pima Indians Diabetes

In [ ]:
estimator = pipeline_neighbors2


weights = ["uniform", "distance"]
n_neighbors = [5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

k_neighbors_clf2 = utils.optimize_params(estimator,
                                        X2, y2, cv,scoring=scoring,refit=refit,
                                        kneighborsclassifier__weights=weights,
                                        kneighborsclassifier__n_neighbors=n_neighbors)

Como podemos observar, se ha obtenido con 9 vecinos, este es un valor intermedio y esperado dentro del intervalo que encuentra un equilibrio adecuado entre sesgo y varianza.

También podemos observar que se ha escogido una estrategia de pesos uniformes, esto implica que los outliers (algunos de ellos han sido imputados en el pipeline) no son de especial relevancia en la generación del modelo.

La mejor tasa de acierto obtenida es de 0.74 y el mejor recall es de 0.52.

### Árboles de decisión

Continuemos con los criterios de los árboles de decisión. Aunque a priori los más destacados serían la profundidad y el criterio, otros hiperparámetros como el mínimo/máximo número de ejemplos por hoja o el valor de ccp_alpha también nos darán juego.

* `criterion`: es el criterio que evalúa la calidad de una partición, mediante su entropía o el criterio gini.
* `max_depth`: la profundidad del árbol. No es interesante probar valores muy bajos, ya que sabemos que el `Zero-R` y el `1-R` no suelen ser buenos clasificadores.
* `ccp_alpha`: valor que controla que las particiones no suponen más coste del que se obtendría de podar dicha rama.
* `min_samples_leaf`: mínimo de ejemplos que debe haber en una hoja. Puede ser un indicativo de que no se está produciendo un sobreajuste.


Ahora, sin embargo, nos enfrentamos a un pequeño problema, y es que si queremos probar todos estos hiperparámetros con un rango más o menos amplio, el tiempo de entrenamiento crecerá mucho. Por lo tanto, deberemos de escoger cuáles son aquellos hiperparámetros que creemos que aportan mayor importancia al clasificador, o por el contrario reducir los rangos de valores que queremos probar. 

En nuestro caso, y al tratarse de un clasificador que no tarda mucho en entrenarse, vamos a probar todos los hiperparámetros en un rango moderado de valores.

### Breast Cancer Wisconsin

In [ ]:
estimator = clone(pipeline_trees)

criterion = ["gini", "entropy"]
max_depth = [3, 4, 5, 6, 7]
ccp_alpha = [0.0, 0.1, 0.2, 0.3, 0.4]
min_samples_leaf = [6, 7, 8]

decision_tree_clf = utils.optimize_params(estimator,
                                         X, y, cv,scoring=scoring,refit=refit,
                                         decisiontreeclassifier__criterion=criterion,
                                         decisiontreeclassifier__max_depth=max_depth,
                                         decisiontreeclassifier__min_samples_leaf=min_samples_leaf,
                                         decisiontreeclassifier__ccp_alpha=ccp_alpha)

Podemos ver que nos quedamos con una pre-poda, las hojas no tendrán menos de 6 instancias y los árboles serán de profunidad 4. Esta profundidad es pequeña teniendo en cuenta que estamos trabajando con casi 600 instancias, por lo que se podría haber esperado un tipo de árbol más profundo. Esto es buena señal, porque quiere decir que nuestros árboles generalizan bien. Además, nos aseguramos de que no haya hojas con solo un ejemplo, puesto que el mínimo será 6. En cuanto a la calidad de las particiones, se utilizará la entropía.

La mejor tasa de acierto obtenida es de 0.93 y el mejor recall es de 0.9.

### Pima Indians Diabetes

In [ ]:
estimator = clone(pipeline_trees2)

criterion = ["gini", "entropy"]
max_depth = [3, 4, 5, 6, 7]
ccp_alpha = [0.0, 0.1, 0.2, 0.3, 0.4]
min_samples_leaf = [6, 7, 8]

decision_tree_clf2 = utils.optimize_params(estimator,
                                          X2, y2, cv,scoring=scoring,refit=refit,
                                          decisiontreeclassifier__criterion=criterion,
                                          decisiontreeclassifier__max_depth=max_depth,
                                          decisiontreeclassifier__min_samples_leaf=min_samples_leaf,
                                          decisiontreeclassifier__ccp_alpha=ccp_alpha)

Como podemos observar, se ha seleccionado como el mejor modelo al entrenado con el criterio 'gini', pre-poda, una máxima profundidad de 4 y un número mínimo de instancias por hoja de 7.

Este número mínimo de instancias por hoja nos puede servir como indicativo de que gracias a la máxima profundidad relativamente pequeña para el número de instancias de la base de datos y la prepoda, se ha evitado el fenómeno de sobreajuste.

La mejor tasa de acierto obtenida es de 0.768 y el mejor recall es de 0.69.

### Adaptative Boosting (AdaBoost)

Sigamos con AdaBoost. Este ensemble se apoyará en árboles de decisión, por lo que le pasaremos los parámetros que ya hemos calculado previamente para que el ensemble funcione aún mejor. No obstante, tenemos que tener en cuenta que estos ensembles ya empiezan a tardar más en entrenarse que otros modelos más simples, osea que tenemos que recordar que la complejidad termporal puede aumentar mucho si nos pasamos con los parámetros. En nuestro caso, el número mínimo de ejemplos por hoja lo vamos a omitir para que el entrenamiento no se demore demasiado.

En cuanto a los parametros del AdaBoost que vamos a optimizar:

* `learning_rate`: tasa de aprendizaje de cada clasificador aprendido. Hemos decidido utilizar un intervalo de números cercanos a 0 puesto que al trabajar con modelos muy simples, debemos adecuar cuánto van a contribuir al modelo existente.
* `base_estimator`: es el clasificador con el que comenzará el ensemble, el cuál será un árbol de decisión. Realmente este clasificador no lo vamos a optimizar sino que va a ser el usado por el ensemble.


### Breast Cancer Wisconsin

In [ ]:
estimator = adaboost_model

# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
learning_rate = [0.05, 0.1, 0.2, 0.3, 1]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]

adaboost_clf = utils.optimize_params(estimator,
                                    X, y, cv,scoring=scoring,refit=refit,
                                    base_estimator=base_estimator,
                                    learning_rate=learning_rate,
                                    base_estimator__criterion=criterion,
                                    base_estimator__max_depth=max_depth,
                                    base_estimator__ccp_alpha=ccp_alpha)

Podemos ver que los parámetros de los árboles de decisión han cambiado con respecto a los que vimos anteriormente, usando la entropía, profundidad 3 y un ccp de 0.1. En cuanto a la tasa de aprendizaje, vemos que ha optado por 0.2, valor bastante cercano al 0, por lo que la contribución de los modelos será pequeña, por lo que se requerirán muchas iteraciones.

La mejor tasa de acierto obtenida es de 0.97 y el mejor recall es de 0.94.

### Pima Indians Diabetes

In [ ]:
estimator = pipeline_adaboost2

# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
learning_rate = [0.1, 0.2, 0.3, 1]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]

adaboost_clf2 = utils.optimize_params(estimator,
                                     X2, y2, cv,scoring=scoring,refit=refit,
                                     adaboostclassifier__base_estimator=base_estimator,
                                     adaboostclassifier__learning_rate=learning_rate,
                                     adaboostclassifier__base_estimator__criterion=criterion,
                                     adaboostclassifier__base_estimator__max_depth=max_depth,
                                     adaboostclassifier__base_estimator__ccp_alpha=ccp_alpha)

Podemos observar que en el mejor modelo se ha obtenido una profundidad del árbol estimador base de 1, esto es debido a que AdaBoost toma como base modelos muy simples con un gran sesgo que va reduciendo iteración a iteración.
En cuanto al mejor criterio de división para el resto de mejores hiperparámetros seleccionados, a diferencia del árbol de decisión obtenido en el apartado anterior, en este caso de ha optado por la entropía.


El único parámetro que afecta directamente al ensemble (y no a los modelos que lo componen) es el learning_rate, que indica el grado de contribución de cada modelo al modelo final.
En este caso ese grado de contribución se ha fijado a 0.2, esto era esperable debido a la simplicidad y gran sesgo de los árboles base.

La mejor tasa de acierto obtenida es de 0.71 y el mejor recall es de 0.67.

### Bootstrap Aggregating (Bagging)

Vayamos ahora con el ensemble Bootstrap Aggregating. De nuevo, el estimador base será un árbol de decisión, por lo que tendremos que calibrar sus hiperparámetros.

### Breast Cancer Wisconsin

In [ ]:
estimator = bagging_model
# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]

bagging_clf = utils.optimize_params(estimator,
                                   X, y, cv,scoring=scoring,refit=refit,
                                   base_estimator=base_estimator,
                                   base_estimator__criterion=criterion,
                                   base_estimator__max_depth=max_depth,
                                   base_estimator__ccp_alpha=ccp_alpha
                                  )

En este caso podemos ver que los árboles utilizados utilizarán el criterio gini para sus particiones, y el resto de hiperparámetros se mantienen igual. No obstante, el rendimiento del estimador es algo inferior a AdaBoost en términos de accuracy, pero mejor en cuanto a su recall.

La mejor tasa de acierto obtenida es de 0.94 y el mejor recall es de 0.98.

### Pima Indians Diabetes

In [ ]:
estimator = pipeline_bagging2
# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]

bagging_clf2 = utils.optimize_params(estimator,
                                    X2, y2, cv,scoring=scoring,refit=refit,
                                    baggingclassifier__base_estimator=base_estimator,
                                    baggingclassifier__base_estimator__criterion=criterion,
                                    baggingclassifier__base_estimator__max_depth=max_depth,
                                    baggingclassifier__base_estimator__ccp_alpha=ccp_alpha
                                   )

Como podemos observar, en este caso el estimador base seleccionado es un árbol de profundidad 1 con pre-poda y criterio de división basado en la entropía, esto no era de esperar ya que en este caso, y a diferencia de AdaBoost, se seleccionan clasificadores *strong learners* con un mayor poder predictivo cuyo exceso de varianza deberá ir reduciéndose mediante agregación.

La mejor tasa de acierto obtenida es de 0.73 y el mejor recall es de 0.60.

### Random Forest

Continuemos con el ensemble Random Forests. Al estar basado en árboles de decisión, de nuevo tendremos que calibrar algunos hiperparámetros que ya hemos visto, como el criterio de medida de las particiones. Veamos cuáles vamos a optimizar:

* `n_estimators`: se trata del número de árboles que creará el ensemble. Teóricamente, cuantos más haya, mejor. Sin embargo, puesto que no tenemos poder computacional ilimitado, deberíamos elegir el número que maximice el score con el menor número de árboles posibles.
* `criterion`: es el criterio que evalúa la calidad de una partición, mediante su entropía o el criterio gini.
* `max_features`: máximo número de características a tener en cuenta para encontrar la mejor partición. Probaremos la raíz cuadrada y el logaritmo en base 2.

### Breast Cancer Wisconsin

In [ ]:
estimator = random_forest_model

n_estimators = [50, 100, 150, 200]
criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]

random_forest_clf = utils.optimize_params(estimator,
                                         X, y, cv,scoring=scoring,refit=refit,
                                         n_estimators=n_estimators,
                                         criterion=criterion,
                                         max_features=max_features)

El criterio, de nuevo, será la entropía. Por otra parte, el número de características será la raíz cuadrada del total. Por último, podemos ver que el número de árboles que se crea es 150, y no 200, que es el número más grande que le hemos dado al estimador. 

Como podemos ver por los resultados, tanto 150 como 200 árboles dan como resultado un score y un recall prácticamente igual, siendo el resto de hiperparámetros idénticos. Por lo tanto, es extraño que el rendimiento del clasificador que utiliza 200 árboles sea inferior.

La mejor tasa de acierto obtenida es de 0.96 y el mejor recall es de 0.94.

### Pima Indians Diabetes

In [ ]:
estimator = pipeline_rf2

n_estimators = [50, 100, 150, 200]
criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]

random_forest_clf2 = utils.optimize_params(estimator,
                                          X2, y2, cv,scoring=scoring,refit=refit,
                                          randomforestclassifier__n_estimators=n_estimators,
                                          randomforestclassifier__criterion=criterion,
                                          randomforestclassifier__max_features=max_features)

Podemos observar que al igual que en el caso anterior, el criterio de división será la entropía, y el número de características será la raíz cuadrada del total.

En cuanto al número de estimadores, en este caso se ha seleccinado el mayor número posible dentro del intervalo (200).

La mejor tasa de acierto obtenida es de 0.76 y el mejor recall es de 0.59.

### Gradient Tree Bosting (Gradient Boosting)

Gradient Tree Bosting, al ser una versión del algoritmo de Boosting, tendrá hiperparámetros similares. No obstante, alguno de ellos se diferencia:

* `criterion`: en este caso, en vez de entropía o gini, el criterio será el error cuadrático medio o la mejora de Friedman de dicho error cuadrático medio.

### Breast Cancer Wisconsin

In [ ]:
estimator = gradient_boosting_model

learning_rate = [0.01, 0.05, 0.1]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]

gradient_boosting_clf = utils.optimize_params(estimator,
                                             X, y, cv,scoring=scoring,refit=refit,
                                             learning_rate=learning_rate,
                                             criterion=criterion,
                                             max_depth=max_depth,
                                             ccp_alpha=ccp_alpha)

Podemos ver que, como en otros clasificadores, se utilizará la pre-poda. En cuanto al resto de hiperparámetros, el criterio será el error cuadrático medio d Friedman, así como una tasa de aprendizaje de 0.1.  Por último, vemos que los árboles usados son de profundidad 1.

La mejor tasa de acierto obtenida es de 0.96 y el mejor recall es de 0.94.

### Pima Indians Diabetes

In [ ]:
estimator = pipeline_gradient2

learning_rate = [0.01, 0.05, 0.1]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]

gradient_boosting_clf2 = utils.optimize_params(estimator,
                                              X2, y2, cv,scoring=scoring,refit=refit,
                                              gradientboostingclassifier__learning_rate=learning_rate,
                                              gradientboostingclassifier__criterion=criterion,
                                              gradientboostingclassifier__max_depth=max_depth,
                                              gradientboostingclassifier__ccp_alpha=ccp_alpha)

En este caso se puede observar que hemos obtenido árboles algo menos complejos (profundidad 2 con pre-poda) como mejores clasificadores base que los obtenidos en bagging, con un learning_rate bastante reducido (0.1). Además, se ha seleccionado la mejora de Friedman sobre el error cuadrático medio como criterio de división.

La mejor tasa de acierto obtenida es de 0.76 y el mejor recall es de 0.62.

### Histogram-Based Gradient Boosting (Histogram Gradient Boosting)

Finalmente, veamos la versión mejorada mediante histogramas de Gradient Boosting. En este ensemble, maximizaremos los siguientes hiperparámetros:

* `learning_rate`: en este caso, utilizaremos valores cercanos a cero.
* `max_leaf_nodes`: número máximo de hojas que tendrán los árboles.

### Breast Cancer Wisconsin

In [ ]:
estimator = hist_gradient_boosting_model

learning_rate = [0.01, 0.02, 0.03, 0.04, 0.05]
max_leaf_nodes = [15, 31, 65, 127]

hist_gradient_boosting_clf = utils.optimize_params(estimator,
                                                  X, y, cv,scoring=scoring,refit=refit,
                                                  learning_rate=learning_rate,
                                                  max_leaf_nodes=max_leaf_nodes)

En el caso del último ensemble de todos, vemos que el número de hojas por árbol es la más pequeña, por lo que los árboles usados no son demasiado grandes. En cuanto a la tasa de aprendizaje, es del 0.05.

La mejor tasa de acierto obtenida es de 0.96 y el mejor recall es de 0.94.

### Pima Indians Diabetes

In [ ]:
estimator = pipeline_h_gradient2

learning_rate = [0.01, 0.02, 0.03, 0.04, 0.05]
max_leaf_nodes = [15, 31, 65, 127]

hist_gradient_boosting_clf2 = utils.optimize_params(estimator,
                                                   X2, y2, cv,scoring=scoring,refit=refit,
                                                   histgradientboostingclassifier__learning_rate=learning_rate,
                                                   histgradientboostingclassifier__max_leaf_nodes=max_leaf_nodes,
                                                   )

Podemos observar que se han obtenido árboles simples (15 nodos por hoja), y una tasa de aprendizaje reducida (0.04).

La mejor tasa de acierto obtenida es de 0.75 y el mejor recall es de 0.61.

## 5. Construcción y validación del modelo final

Ya tenemos todos los clasificadores optimizados, por lo que ahora tendremos que elegir el mejor de todos. Ya que es complicado observar los resultados en el apartado anterior, utilizaremos otro grid, al que le pasaremos los clasificadores que acabamos de construir, y construiremos una tabla que nos permita divisar los datos de manera cómoda.

### Wisconsin Breast Cancer

In [ ]:
estimators = {
   "Nearest neighbors": k_neighbors_clf,
   "Decision tree": decision_tree_clf,
   "AdaBoost": adaboost_clf,
   "Bagging": bagging_clf,
   "Random Forests": random_forest_clf,
   "Gradient Boosting": gradient_boosting_clf,
   "Histogram Gradient Boosting": hist_gradient_boosting_clf
}

In [ ]:
utils.evaluate_estimators(estimators, X_test, y_test)

Puesto que se trata de un problema de clasificación de diagnóstico, y que sabemos que la muestra está sesgada, aunque la métrica del accuracy pueda darnos un indicio de lo bueno que puede llegar a ser un clasificador, nosotros nos vamos a quedar con aquellos que nos den el mejor recall. En nuestro caso, podemos ver como el clasificador que mayor recall nos da es AdaBoost, con un 95% de recall, seguido de Histogram Gradient Boosting y los Árboles de decisión. 

Siendo tan importante que las clasificaciones sean correctas, nos deberíamos quedar con `AdaBoost` porque es el que mayor fiabilidad nos va a dar, aunque sea un modelo que tarda bastante en aprenderse debido a las iteraciones que necesita. No obstante, si nos encontraramos en un problema donde el tiempo es un factor mucho más relevante, podríamos quedarnos con los árboles de decisión, que a pesar de tener un recall algo menor, se aprenden mucho más rápido.

Como era de esperar, los ensembles dan unos muy buenos resultados en cuando al accuracy, aunque los árboles de decisión no se quedan cortos. No obstante, como estamos buscando maximizar otras métricas, podemos ver que, a pesar del alto coste computacional y temporal que suponen, no nos son de tanta utilidad.

### Pima Indians Diabetes

In [ ]:
estimators = {
    "Nearest neighbors": k_neighbors_clf2,
    "Decision tree": decision_tree_clf2,
    "AdaBoost": adaboost_clf2,
    "Bagging": bagging_clf2,
    "Random Forests": random_forest_clf2,
    "Gradient Boosting": gradient_boosting_clf2,
    "Histogram Gradient Boosting": hist_gradient_boosting_clf2
}

In [ ]:
utils.evaluate_estimators(estimators, X2_test, y2_test)

Podemos ver que al contrastar los modelos con el conjunto de test, la mayor tasa de acierto se ha obtenido para Gradient Boosting, pero dado que el conjunto de datos no está balanceado y se trata de un problema de diagnóstico, conviene fijarse en la métrica del recall.

El mejor valor de recall se ha obtenido en el árbol de decisión (0.69), por lo que, aunque la tasa de acierto es menor, el número de casos positivos clasificados como negativos es menor, por lo que este modelo resulta más adecuado para el diagnóstico.

Como **conclusión final**, podemos ver que, efectivamente, los ensembles son una buena herramienta para la clasificación a costa de su alto tiempo de entrenamiento. Además, si nos fijamos en los valores de las métricas antes y después de la optimización de los parámetros, vemos que en el peor caso se mantienen, mientras que otros casos, como el caso de los vecinos más cercanos, el valor del recall pasa del 0.78 al 0.89. Esto refuerza la importancia de dedicarle tiempo a la selección de hiperparámetros de los modelos, así como a la comprensión de los propios hiperparámetros, puesto que podemos ajustar los valores necesarios para que los algoritmos funcionen mucho mejor.

## Estudio de un kernel

En nuestro caso, vamos a estudiar un kernel que trata sobre la base de datos de Titanic, donde tras realizar un preprocesamiento de los datos, se entrena un KNN para clasificar si un pasajero del Titanic sobrevivirá o no.

El enlace al kernel se encuentra aquí: [Predicting Titanic Survival using KNN](https://www.kaggle.com/abhishekchhibber/predicting-titanic-survival-using-knn)